In [91]:
import pandas as pd
import numpy as np

rolling_df = pd.read_csv("matches (rolling stats).csv")         ## Load the data

training_data = rolling_df[rolling_df['date'] < '2024-06-01']   ## Training data from before date
testing_data = rolling_df[rolling_df['date'] > '2024-06-01']    ## Testing data after date

prediction_metrics = ['venue code', 'gf_rolling', 'sot_rolling', 'xg_strength']
#prediction_metrics = ['venue code', 'xg_strength']
#prediction_metrics = ['venue code',  'gf_rolling', 'sot_rolling']
#prediction_metrics = ['gf_rolling']
#prediction_metrics = ['xg_ratio']

In [92]:
import scipy as sc

#################    Sigmoid function puts results between 0 - 1    #################
def sigmoid(x):
    S = 1 / (1+np.exp(-x))
    return(S)

#################    Softmax function noramlises results between 0 - 1    #################
def softmax(x):
    x = np.array(x, dtype=np.float64)
    sigma = np.exp(x) / sum(np.exp(x))
    return(sigma)

#################    Relu function to set negative nodes to 0    #################
def relu(z):          
    out = np.maximum(0, z)
    return(out)

#################    Relu derivative function to produce binary nodes    #################
def relu_derivative(y):
    out = (y>0).astype(float)     ## if positive set to 1
    return(out)
    
#################    Leaky Relu  function so small negative values can stay in the model    #################
def leaky_relu(x, alpha=0.01):
    y = np.where(x > 0, x, alpha * x)   
    return(y)                        



In [93]:
def layer_initialisation(number_of_hidden_layers, layer_sizes, input_size, output_size):

    if len(layer_sizes) != number_of_hidden_layers:
        print(f'layer sizes:{len(layer_sizes)}', f'number of layers:{number_of_hidden_layers}')
        raise Exception("layer sizes must equal number of hidden layers")
        
    layer_sizes.append(output_size)      ## adds the output number of nodes to list of layer sizes
    
    ############# Creating initial weighting matrixes and biases ###############
    
    weighting_dict = {}                                             ## 
    bias_dict = {}                                               ## holds all the biases
    for i in range(0, number_of_hidden_layers+1):                ## iterates through each possible layer connection
    
        if i == 0:          ## creating the first layer
            xavier_weight_mat = np.random.uniform(-1/np.sqrt(input_size),1/np.sqrt(input_size), (layer_sizes[i], input_size))    ## xavier distribution    https://www.geeksforgeeks.org/xavier-initialization/
        elif i < number_of_hidden_layers+2:  ## creating the middle layers
            xavier_weight_mat = np.random.uniform(-1/np.sqrt(layer_sizes[i]),1/np.sqrt(layer_sizes[i]), (layer_sizes[i], layer_sizes[i-1]))
       
        bias = np.zeros((layer_sizes[i],1))
        weighting_dict[f'layer_{i+1}'] = xavier_weight_mat      ## Appends the rand matrix to the layers dictionary
        bias_dict[f'bias_{i+1}'] = bias
    
    return(weighting_dict, bias_dict)

def node_activations(weighting_dict, bias_dict, training_data_row, prediction_metrics):

    input_vector = training_data_row[prediction_metrics]
    
    pre_node_dict = {}                                 ## Creates empty dictionary for the pre node layers
    act_node_dict = {}                                 ## Creates empty dictionary for the active node layers
    counter = 1                                        ## Initialsies counter for calling dictionary items

    pre_node_dict[f'node_0'] = input_vector  
    
    for item, weighting in weighting_dict.items():     ## item is the name of the dictionary item, value is the matrix assigned to the item
       
        if counter == 1:                               ## this deals with creating the first layer from input layer
            pre_node = weighting @ input_vector        ## create the first layer 
            pre_node += bias_dict[f'bias_{counter}'].flatten()   ## add the bias to the first layer of pre activated nodes
            
            act_node = relu(pre_node)            ## activate the nodes
            
        else:
            pre_node = weighting @ np.array(act_node_dict[f'node_{counter-1}']).flatten()    ## create the next layer using previous activation, sometimes ould create nested array so convert to array then flatten
            pre_node += bias_dict[f'bias_{counter}'].flatten()           ## add the bias to the next layer
    
            if counter == len(weighting_dict):
                act_node = softmax(pre_node)           ## if final node use softmax activation
            else:
                act_node = relu(pre_node)              ## otherwise use relu activation

        pre_node_dict[f'node_{counter}'] = pre_node           ## Append each pre actiation node layer to dictionary
        act_node_dict[f'node_{counter}'] = act_node           ## Append each activated node layer to dictionary
    
        counter += 1                                  ## Increment counter for dictionary extractions
    
    return(pre_node_dict, act_node_dict, bias_dict)


In [94]:
def backpropagation(training_data_row, weighting_dict, bias_dict, pre_node_dict, act_node_dict, output_size, learning_rate=0.0001):

    blank = np.zeros((output_size,1))                 ## array of zeros of same size as output array
    true = blank.copy()                               ## copy array of zeros
    true[int(training_data_row['gf']), 0] = 1         ## creates vector of actual goals scored

    ######################################### updating the weights #########################################
    
    alpha = learning_rate             ## alpha is set to the learning rate
    count = len(weighting_dict)       ## initiasing count at max value
    output = list(act_node_dict.values())[-1]
    
    updated_weighting_dict = {}
    updated_bias_dict = {}
    
    for i in range(count,0,-1):
        
        if count == len(weighting_dict):           ## for the inital backpropagation from output layer
            dl_dz =  output - true.flatten()       ## gradient wrt logits for softmax + CE loss
        else:
     
            dl_dz = np.transpose(weighting_dict[f'layer_{count+1}']) @ dl_dz              ## first step of loss 
            dl_dz = np.multiply(dl_dz , relu_derivative(pre_node_dict[f'node_{count}']))  ## second step of element wise                         

        if dl_dz.ndim == 1:                              ## if only 1 dimensional               
            dl_dz = np.expand_dims(dl_dz, axis=1)        ## makes sure that its of the form (x,1) rather than (8,)

        dl_db = dl_dz                 ##  biases independent of input activation so same as loss to pre node values
        pre_node_dict[f'node_{count-1}'] = np.expand_dims(pre_node_dict[f'node_{count-1}'], axis =1)   ## dealing with array formatting
        
        dummy_weighting = weighting_dict[f'layer_{count}'] - alpha * (dl_dz @ np.transpose(pre_node_dict[f'node_{count-1}']) )   ## W = W - a*(dL/dW) = W - a*loss*input_into_node
        dummy_bias = bias_dict[f'bias_{count}'] - alpha * dl_db    ## Create new updated bias
    
        updated_weighting_dict[f'layer_{count}'] = dummy_weighting     ## update the old weighting with the new weighting
        updated_bias_dict[f'bias_{count}'] = dummy_bias               ## update the old bias with the new bias
 
        count -= 1    ## increment the count by -1 to
    
    weighting_dict.update(updated_weighting_dict)     ## update the old weighting dictionary with the new dictionary
    bias_dict.update(updated_bias_dict)               ## update the old bias dictionary with the new dictionary
    
    
    return(weighting_dict, bias_dict)

In [95]:
## Each metric in the first layer is multiplied by the corresponding component for a weighting veector for each node ##
## If we have 5 metrics and 10 first layer nodes then we need 10 vectors of size 5 ##
## The first component of each of these 10 metrics contributes to the magnitude fo the first node, and so on ##
## We need to start the initial vectors in some random state, limiting magnitudes of components to prevent vanishing/exploding ##


def neural_network_processing(training_data, prediction_metrics, number_of_hidden_layers, layer_sizes, output_size, prev_weights=None, prev_biases=None):

    input_size = len(prediction_metrics)

    if (prev_weights == None) and (prev_biases == None):          ## run initilisation if no existing weights exist
        weighting_dict, bias_dict = layer_initialisation(number_of_hidden_layers, layer_sizes, input_size, output_size)
    else:                
        weighting_dict, bias_dict = prev_weights, prev_biases    

    for index, row in training_data.iterrows():     ## iterate through every row in the training data 
    
        training_data_row = row  ## only need the prediciton metrics from each row    
        pre_node_dict, act_node_dict, bias_dict = node_activations(weighting_dict, bias_dict, training_data_row, prediction_metrics) ## activate the nodes
        weighting_dict, bias_dict = backpropagation(training_data_row, weighting_dict, bias_dict, pre_node_dict, act_node_dict, output_size, learning_rate=0.1) ## use backpropagation to update the weightings    

    return(weighting_dict, bias_dict)



In [96]:
def forward_process(weighting_dict, bias_dict, testing_data_row, prediction_metrics):

    input_vector = testing_data_row[prediction_metrics]
    
    pre_node_dict = {}                                 ## Creates empty dictionary for the pre node layers
    act_node_dict = {}                                 ## Creates empty dictionary for the active node layers
    counter = 1                                        ## Initialsies counter for calling dictionary items

    pre_node_dict[f'node_0'] = input_vector  
    
    for item, weighting in weighting_dict.items():     ## item is the name of the dictionary item, value is the matrix assigned to the item
        
        if counter == 1:                               ## this deals with creating the first layer from input layer
            pre_node = weighting @ input_vector        ## create the first layer 
            pre_node += bias_dict[f'bias_{counter}'].flatten()   ## add the bias to the first layer of pre activated nodes
            
            act_node = relu(pre_node)            ## activate the nodes
            
        else:
            pre_node = weighting @ np.array(act_node_dict[f'node_{counter-1}']).flatten()    ## create the next layer using previous activation, sometimes ould create nested array so convert to array then flatten
            pre_node += bias_dict[f'bias_{counter}'].flatten()           ## add the bias to the next layer
    
            if counter == len(weighting_dict):
                act_node = softmax(pre_node)           ## if final node use softmax activation
            else:
                act_node = relu(pre_node)              ## otherwise use relu activation

    
        pre_node_dict[f'node_{counter}'] = pre_node           ## Append each pre actiation node layer to dictionary
        act_node_dict[f'node_{counter}'] = act_node           ## Append each activated node layer to dictionary

        counter += 1                                  ## Increment counter for dictionary extractions

    return(list(act_node_dict.values())[-1])

In [97]:
number_of_hidden_layers = 3           ## number of hidden layers you want in network
layer_sizes = [10, 10, 10]               ## number of nodes in the each layer
prediction_metrics = ['venue code', 'gf_rolling', 'sot_rolling', 'xg_strength']
max_goals = 7
output_size = max_goals + 1

###############################   Trainging loop with epochs  #################################

epochs = 30
for epoch in range(0,epochs):
    
    if epoch == 0:                  ## if first loop then run with no previous weighting or biases
        weighting_dict, bias_dict = neural_network_processing(training_data, prediction_metrics, number_of_hidden_layers, layer_sizes,  #### first training process
                                                            output_size, prev_weights=None, prev_biases=None)
    else:                           ## otherwise loop back using previous weightings and biases
        weighting_dict, bias_dict = neural_network_processing(training_data, prediction_metrics, number_of_hidden_layers, layer_sizes, 
                                                            output_size, prev_weights=weighting_dict, prev_biases=bias_dict)
    
final_weighting_dict = weighting_dict
final_bias_dict = bias_dict

In [98]:
number_of_hidden_layers = 3           ## number of hidden layers you want in network
layer_sizes = [10, 10, 10]               ## number of nodes in the each layer
prediction_metrics = ['venue code', 'gf_rolling', 'sot_rolling', 'xg_strength']
max_goals = 7
output_size = max_goals + 1


correct = 0
incorrect = 0
for index, row in testing_data.iterrows():     ## iterate through every row in the training data 
        
    testing_data_row = row      ## only need the prediciton metrics from each row
    
    prediction_vector = forward_process(final_weighting_dict, final_bias_dict, testing_data_row, prediction_metrics)

    predicted_goals = np.argmax(prediction_vector)
    actual_goals = testing_data_row['gf']

    #print(f'predicted:{predicted_goals}    actual:{actual_goals}')
    #print(prediction_vector)
    
    if int(actual_goals) == predicted_goals:
        correct += 1
    else:
        incorrect += 1
print(f'correct:{correct}   incorrect:{incorrect}')
print('acc', correct/(len(testing_data)))

correct:92   incorrect:248
acc 0.27058823529411763


In [99]:
weighting_dict

{'layer_1': array([[-0.05255854664267912, 0.0039466279510695455, 0.13597706087624634,
         0.14537694395032283],
        [0.23227731164840162, 0.17532107927484294, 0.1254219184237789,
         0.13695210916036257],
        [0.07659914030892996, 0.3115138780596014, 0.20620226178056703,
         0.2812294668578852],
        [-0.4962515165458805, -0.1860339528421563, -0.03100820653145913,
         0.03482704909290382],
        [-0.27014358798580695, 0.4734169028226971, -0.2292021623797349,
         -0.13275707163995476],
        [0.44656345889622456, 0.027957179745697655, -0.3370480318468005,
         0.23466158405887277],
        [0.43833567707175936, 0.12825766751398554, -0.3636421963168344,
         0.0009173090612032011],
        [0.6586711838889949, -0.20003731709232292, 0.19650271018012797,
         0.38185835185731865],
        [0.018334069581853463, 0.3363908430033967, -0.23184882400724272,
         -0.04003123433817035],
        [-0.4854384252829645, -0.5252913614593304, 0.14

In [100]:
from sklearn.neural_network import MLPClassifier

MLP = MLPClassifier(hidden_layer_sizes=(10,10), max_iter=1000)
MLP.fit(training_data[prediction_metrics], training_data['gf'])


C:\Users\joshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000)

In [101]:
predictions = MLP.predict(testing_data[prediction_metrics])

In [102]:
from sklearn.metrics import classification_report

predictions


print(classification_report( (testing_data['gf']) , predictions) )

              precision    recall  f1-score   support

         0.0       0.28      0.29      0.28        76
         1.0       0.35      0.44      0.39       111
         2.0       0.30      0.38      0.34        92
         3.0       0.00      0.00      0.00        39
         4.0       0.00      0.00      0.00        13
         5.0       0.00      0.00      0.00         6
         6.0       0.00      0.00      0.00         2
         7.0       0.00      0.00      0.00         1

    accuracy                           0.31       340
   macro avg       0.12      0.14      0.13       340
weighted avg       0.26      0.31      0.28       340



C:\Users\joshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\joshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\joshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [103]:
x = testing_data['gf'] - predictions
#x.ndims(1,)
x = np.array(x)
correct = len(np.where(x==0)[0])
total = len(testing_data)

print('acc',correct/total)

acc 0.31176470588235294


In [37]:
import json
'''
weightings_serializable = {key: value.tolist() for key, value in final_weighting_dict.items()}
biases_serializable = {key: value.tolist() for key, value in final_bias_dict.items()}


with open("final weightings.json", "w") as f:
    json.dump(weightings_serializable, f)

with open("final biases.json", "w") as f:
    json.dump(biases_serializable, f)'''


'\nweightings_serializable = {key: value.tolist() for key, value in final_weighting_dict.items()}\nbiases_serializable = {key: value.tolist() for key, value in final_bias_dict.items()}\n\n\nwith open("final weightings.json", "w") as f:\n    json.dump(weightings_serializable, f)\n\nwith open("final biases.json", "w") as f:\n    json.dump(biases_serializable, f)'

In [56]:
with open("final weightings.json", "r") as f:
    weights_loaded_data = json.load(f)

with open("final biases.json", "r") as f:
    biases_loaded_data = json.load(f)


weights_loaded_data = {key: np.array(value) for key, value in weights_loaded_data.items()}        
biases_loaded_data = {key: np.array(value) for key, value in biases_loaded_data.items()}    


#weights_loaded_data

In [55]:
#final_weighting_dict